In [1]:
import numpy as np
import json

In [2]:
corpus = np.load('corpus.npy')

In [22]:
k = 6
objective = list()
centers = corpus[np.random.choice(np.arange(corpus.shape[0]), k, replace=False)]
groups = np.array([np.argmin([np.linalg.norm(center - x) for center in centers]) for x in corpus])
objective.append(
    np.array([np.linalg.norm(corpus[i] - centers[groups[i]]) for i in range(corpus.shape[0])]).sum()
)

i = 0
while True:
    centers = np.array([corpus[groups == g].mean(axis=0) for g in range(k)])
    groups = np.array([np.argmin([np.linalg.norm(center - x) for center in centers]) for x in corpus])
    objective.append(
        np.array([np.linalg.norm(corpus[i] - centers[groups[i]]) for i in range(corpus.shape[0])]).mean()
    )
    if i > 2 and ((objective[-1] - objective[-2]) + (objective[-2] - objective[-3])) < 1.0:
        break
    i += 1

In [23]:
objective[-1]

np.float64(13.196908448848285)

In [24]:
corpus[groups == 0][0]

array([3, 2, 3, ..., 0, 0, 0])

In [25]:
np.save('groups.npy', groups)

Identify the words present in each cluster

In [26]:
words = np.load('words.npy')
corpus = np.load('corpus.npy')
groups = np.load('groups.npy')
articles = list()
with open('articles.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)

How many articles in each cluster

In [27]:
cluster_articles_n = {g: corpus[groups == g].shape[0] for g in range(6)}

In [28]:
cluster_articles_n

{0: 2326, 1: 138, 2: 44, 3: 28, 4: 276, 5: 187}

In [29]:
big_cluster_i = np.array(list(cluster_articles_n.values())).argmax()

In [30]:
cluster_word_histogram = corpus[groups == 0].sum(axis=0)

In [31]:
non_zero_i = np.where(cluster_word_histogram > 0)

In [32]:
cluster_hist_norm = cluster_word_histogram / np.linalg.norm(cluster_word_histogram)

In [33]:
cluster_hist_norm.dtype

dtype('float64')

In [34]:
print(*words[non_zero_i])

body brother strike support state volodymyr friday border russia war federation picture result store day ministry sense administration chief prosecutor reach morning kharkiv defense address child region lady boy missile president october attack show city thursday minister village information evidence round group fleet half fighting blast moscow source storage monitoring analyst fight front ukraine amount term germany action tank intelligence month cover paper chance website force technology design battle week infantry focus toll figure ammunition equipment sent invasion count armor way territory religion airport afternoon madrid ceremony part woman assault suburb police food warrant ticket affiliate authority decision gas time lack maintenance peninsula bfmtv operation sign girlfriend list course magazine david arrest december law order edition briefly name launch enforcement plot flight building march look court terror terrorism officer title responsibility wife newspaper station inve

In [35]:
values_i = np.argsort(cluster_hist_norm)[::-1]

In [36]:
cluster_word_histogram[values_i[0]]

np.int64(1884)

In [37]:
words[values_i]

array(['time', 'world', 'team', ..., 'kate', 'brexit', 'rhetoric'],
      dtype='<U16')

In [38]:
cluster_word_histogram.argmax()

np.int64(99)

In [39]:
words[99]

np.str_('time')

In [80]:
list_of_nodes = list()
for i in range(k):
    nodes = list()
    for g in np.where(groups == i)[0]:
        nodes.append({
                'id': articles[g][1],
                'url': articles[g][2],
                'dist': float(np.linalg.norm(corpus[g] - centers[5])),
                'words': [str(word) for word in words[np.argsort(corpus[g])[::-1][:5]]]
            })
    list_of_nodes.append(nodes)

In [81]:
for nodes in list_of_nodes:
    print(nodes)

[{'id': '10-year-old boy killed in Kharkiv was asleep when Russian missile hit, Ukraine says', 'url': 'https://edition.cnn.com/2023/10/06/europe/10-year-old-ukraine-boy-asleep-when-russia-drone-strike-intl-hnk/index.html', 'dist': 13.644254999342538, 'words': ['kharkiv', 'strike', 'body', 'brother', 'war']}, {'id': 'Russia may have lost up to half of its operational tank fleet in Ukraine, monitoring group says', 'url': 'https://edition.cnn.com/2023/02/09/europe/1000-russian-tanks-destroyed-ukraine-war-intl-hnk-ml/index.html', 'dist': 14.462561823103256, 'words': ['war', 'tank', 'russia', 'equipment', 'invasion']}, {'id': "'Unbelievable' 70-shot rally sends Australian Open fans into raptures", 'url': 'https://edition.cnn.com/2023/01/18/tennis/70-shot-rally-kubler-khachanov-australian-open-spt-intl/index.html', 'dist': 12.94896243959022, 'words': ['rally', 'match', 'record', 'tennis', 'attempt']}, {'id': 'Catch of the year? Aaron Judge crashes through door to make spectacular grab; misse

In [79]:
str(np.str_('c'))

'c'

In [63]:
words[np.argsort(corpus[51])[::-1][:5]]

array(['place', 'support', 'nfl', 'game', 'stand'], dtype='<U16')

In [72]:
i = 2000
corpus[i][np.argsort(corpus[i])[::-1][:5]]

array([8, 6, 4, 4, 4])

In [59]:
corpus[0][254]

np.int64(0)

Sub cluster the articles in the biggest cluster

In [145]:
sub_articles = [articles[i] for i in np.where(groups == 0)[0]]

In [146]:
sub_articles

[['2023-10-07',
  '10-year-old boy killed in Kharkiv was asleep when Russian missile hit, Ukraine says',
  'https://edition.cnn.com/2023/10/06/europe/10-year-old-ukraine-boy-asleep-when-russia-drone-strike-intl-hnk/index.html',
  '10-year-old-ukraine-boy-asleep-when-russia-drone-strike-intl-hnk',
  'europe'],
 ['2023-02-09',
  'Russia may have lost up to half of its operational tank fleet in Ukraine, monitoring group says',
  'https://edition.cnn.com/2023/02/09/europe/1000-russian-tanks-destroyed-ukraine-war-intl-hnk-ml/index.html',
  '1000-russian-tanks-destroyed-ukraine-war-intl-hnk-ml',
  'europe'],
 ['2023-01-18',
  "'Unbelievable' 70-shot rally sends Australian Open fans into raptures",
  'https://edition.cnn.com/2023/01/18/tennis/70-shot-rally-kubler-khachanov-australian-open-spt-intl/index.html',
  '70-shot-rally-kubler-khachanov-australian-open-spt-intl',
  'tennis'],
 ['2023-06-05',
  'Catch of the year? Aaron Judge crashes through door to make spectacular grab; misses next ga

In [123]:
groups

array([0, 0, 4, ..., 0, 0, 0])

In [82]:
sub_corpus = corpus[groups == 0]

In [85]:
# Create the word histogram within the cluster
sub_histogram = sub_corpus.sum(axis=0)

In [89]:
included_words = np.where(sub_histogram >= 20)[0]

In [90]:
len(included_words)

1641

In [92]:
sub_words = words[included_words]

In [97]:
print(corpus.shape)
print(sub_corpus[:,included_words].shape)

(2999, 2035)
(2326, 1641)


In [98]:
sub_corpus = sub_corpus[:,included_words]

In [119]:
kk = 6
sub_objective = list()
sub_centers = sub_corpus[np.random.choice(np.arange(sub_corpus.shape[0]), kk, replace=False)]
sub_groups = np.array([np.argmin([np.linalg.norm(center - x) for center in sub_centers]) for x in sub_corpus])
sub_objective.append(
    np.array([np.linalg.norm(sub_corpus[i] - sub_centers[sub_groups[i]]) for i in range(sub_corpus.shape[0])]).sum()
)

i = 0
while True:
    sub_centers = np.array([sub_corpus[sub_groups == g].mean(axis=0) for g in range(kk)])
    sub_groups = np.array([np.argmin([np.linalg.norm(center - x) for center in sub_centers]) for x in sub_corpus])
    sub_objective.append(
        np.array([np.linalg.norm(sub_corpus[i] - sub_centers[sub_groups[i]]) for i in range(sub_corpus.shape[0])]).mean()
    )
    if i > 2 and ((sub_objective[-1] - sub_objective[-2]) + (sub_objective[-2] - sub_objective[-3])) < 1.0:
        break
    i += 1

In [120]:
{g: sub_corpus[sub_groups == g].shape[0] for g in range(kk)}

{0: 1294, 1: 173, 2: 101, 3: 64, 4: 585, 5: 109}

In [147]:
list_of_sub_nodes = list()
for i in range(kk):
    sub_nodes = list()
    for g in np.where(sub_groups == i)[0]:
        sub_nodes.append({
                'id': sub_articles[g][1],
                'url': sub_articles[g][2],
                'dist': float(np.linalg.norm(sub_corpus[g] - sub_centers[5])),
                'words': [str(word) for word in sub_words[np.argsort(sub_corpus[g])[::-1][:5]]]
            })
    list_of_sub_nodes.append(sub_nodes)

In [148]:
for nodes in list_of_sub_nodes:
    print(nodes)

[{'id': '10-year-old boy killed in Kharkiv was asleep when Russian missile hit, Ukraine says', 'url': 'https://edition.cnn.com/2023/10/06/europe/10-year-old-ukraine-boy-asleep-when-russia-drone-strike-intl-hnk/index.html', 'dist': 11.663296307670706, 'words': ['kharkiv', 'strike', 'body', 'village', 'war']}, {'id': 'Russia may have lost up to half of its operational tank fleet in Ukraine, monitoring group says', 'url': 'https://edition.cnn.com/2023/02/09/europe/1000-russian-tanks-destroyed-ukraine-war-intl-hnk-ml/index.html', 'dist': 12.153737360349488, 'words': ['war', 'tank', 'russia', 'equipment', 'invasion']}, {'id': "'Unbelievable' 70-shot rally sends Australian Open fans into raptures", 'url': 'https://edition.cnn.com/2023/01/18/tennis/70-shot-rally-kubler-khachanov-australian-open-spt-intl/index.html', 'dist': 12.66636238545519, 'words': ['rally', 'match', 'record', 'tennis', 'attempt']}, {'id': 'Aaron Rodgers is starting his darkness retreat this week -- and he might film it', 

In [131]:
articles

[['2023-10-07',
  '10-year-old boy killed in Kharkiv was asleep when Russian missile hit, Ukraine says',
  'https://edition.cnn.com/2023/10/06/europe/10-year-old-ukraine-boy-asleep-when-russia-drone-strike-intl-hnk/index.html',
  '10-year-old-ukraine-boy-asleep-when-russia-drone-strike-intl-hnk',
  'europe'],
 ['2023-02-09',
  'Russia may have lost up to half of its operational tank fleet in Ukraine, monitoring group says',
  'https://edition.cnn.com/2023/02/09/europe/1000-russian-tanks-destroyed-ukraine-war-intl-hnk-ml/index.html',
  '1000-russian-tanks-destroyed-ukraine-war-intl-hnk-ml',
  'europe'],
 ['2023-12-25',
  '2015 Charlie Hebdo Attacks Fast Facts',
  'https://edition.cnn.com/2015/01/21/europe/2015-paris-terror-attacks-fast-facts/index.html',
  '2015-paris-terror-attacks-fast-facts',
  'europe'],
 ['2023-04-01',
  "2023 March Madness: A men's Final Four like no other",
  'https://edition.cnn.com/2023/04/01/sport/2023-march-madness-mens-final-four-preview-spt-intl/index.html'